In [1]:
! pip install transformers
! pip install datasets
!pip install evaluate
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.4 MB/s 
     |████████████████████████████████| 7.6 MB 75.6 MB/s 
     |████████████████████████████████| 163 kB 88.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 7.6 MB/s 
     |████████████████████████████████| 115 kB 84.0 MB/s 
     |████████████████████████████████| 212 kB 87.6 MB/s 
     |████████████████████████████████| 95 kB 6.2 MB/s 
     |████████████████████████████████| 127 kB 86.7 MB/s 
     |████████████████████████████████| 115 kB 83.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully unin

In [2]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModel
from collections import defaultdict
from transformers import get_linear_schedule_with_warmup
import time
import seqeval
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
from seqeval.scheme import IOB2
from seqeval.scheme import IOB1
import evaluate
import re

In [3]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# from sklearn.metrics import f1_score
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from transformers import BertModel, BertTokenizer
import os
from pandas.core.frame import DataFrame
from copy import deepcopy
import logging

In [4]:
import json

In [5]:
from google.colab import drive
drive.mount('/content/drive')
# seqeval = evaluate.load('seqeval')

Mounted at /content/drive


In [6]:
# enumerate dataset, each value is a dict data type, each value has to keys which are tokens and ner_tags
# i extract the tokens and ner_tags from dataset and put them in two lists.
def get_fulldata(dataset):
    tokens = []
    ner_tags = []
    for idx,value in enumerate(dataset):
        tokens.append(value['tokens'])
        ner_tags.append(value['ner_tags'])
    return tokens,ner_tags

In [7]:

def produce_database(tokens,ner_tags,num,label_num): 
    # initialize a label_list which includes 1-279 numbers
    label_list = [x for x in range(1,(label_num+1))] 
    tk = []
    nt = []
    index_save = []#save the index to check if they have been chosed
    
    for i in label_list:
        # 计数器tmp
        tmp = 0
        for idx,value in enumerate(ner_tags):
            #if the label i in the value of ner_tags, add it in the tk list
            if i in value:
                # check if the data has been saved in the index_save list by index
                if idx not in index_save:
                    nt.append(value)
                    tk.append(tokens[idx])
                    tmp = tmp+1
                    index_save.append(idx)
            # if temp == 5, means that there are 5 items of data has been saved, then break the for loop
            if tmp==num:
                
                # print('finished label '+str(i))
                
                break
               # if the data is the last one and tmp is still 0, print(no any data), then break, else print(data is less then 5)
            if(idx == (len(ner_tags)-1)):
                if tmp==0:
                    # print('label '+str(i)+' no any data')
                    break
                # else:
                #     # print('label '+str(i)+' less than '+ str(num))
     #convert it to dataframe and save it as datasets              
    df = pd.DataFrame.from_dict({'tokens':tk,'ner_tags':nt})
    final_dataset = datasets.Dataset.from_pandas(df)
    return  final_dataset,df

In [8]:
import re
import random

# mask number function, input a list and the probablity, traverse the token list and detect number by regular expression, 
# and convert it to a [MASK] token, finaaly return the list
def random_mask_number(probability, tokens_list):

    save = []
    processed_text = []
    for i in range(probability):
        save.append(1)
    for x in range(100 - probability):
        save.append(0)
    for token in tokens_list:
      if re.fullmatch(r"(\d+[\d,.]*)|([,.]\d+)", token):
        a = random.choice(save)
        if a == 0:
          processed_text.append(token)
        if a == 1:
          processed_text.append('[MASK]')
      else:
        processed_text.append(token)
    return processed_text

In [9]:
def data_preprocess(data,max_len):
  # max_length = 200, so remove or pad the tokens and ner_tags to 200
  tokens = []
  ner_tags = []
  for i in range(len(data)):
    if(len(data[i]['tokens'])>max_len): 
      tokens.append(data[i]['tokens'][:max_len])
    else:
      tokens.append(data[i]['tokens'])
    if(len(data[i]['ner_tags'])>max_len):
      ner_tags.append(data[i]['ner_tags'][:max_len])
    else:
      ner_tags.append(data[i]['ner_tags'])
  return tokens,ner_tags

In [10]:
class FewShot_NER(nn.Module):
  def __init__(self,batch_size,dataset_name):
    super().__init__()
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.token_encoder = AutoModel.from_pretrained('yiyanghkust/finbert-tone').to(self.device)
    self.label_encoder = AutoModel.from_pretrained('yiyanghkust/finbert-tone').to(self.device)
    self.dataset_name = dataset_name
    
    
    self.tokenizer = AutoTokenizer.from_pretrained('yiyanghkust/finbert-tone',do_lower_case=True)
    
    self.batch_size = batch_size
    self.label_num,self.idx2tags = self.get_dataset_info()
    self.full_label_name = self.build_label_representation()[1]

  def get_dataset_info(self):
     
    all_label_name = pd.read_csv('/content/drive/MyDrive/latest_dataset/{}/{}.csv'.format(self.dataset_name,self.dataset_name))

    #get dataset label total number
    label_num = len(all_label_name)
    all_label_name = all_label_name.to_dict()
    all_label_name = all_label_name['0']
    idx2tags = all_label_name

    return label_num,idx2tags

    

  def build_label_representation(self):
    label_name = [i for i in self.idx2tags.values()]
    full_label_name = []

    for label in label_name:
      if label == 'O':
        label_rep = 'other'
      elif label[0:2]=='B-':
        label_rep = 'begin {}'.format(' '.join(re.findall('[A-Z][^A-Z]*', label[2:]))).lower()
      elif label[0:2]=='I-':
        label_rep = 'inside {}'.format(' '.join(re.findall('[A-Z][^A-Z]*', label[2:]))).lower()
      full_label_name.append(label_rep)

    tag_max_len = max([len(l) for l in full_label_name])
    

    tag_embeddings = []
    for label in full_label_name:
      input = self.tokenizer.encode_plus(label,return_tensors='pt',padding='max_length',max_length=tag_max_len)
      outputs = self.label_encoder(input_ids=input['input_ids'].to(self.device),attention_mask = input['attention_mask'].to(self.device))
      pooler_output = outputs.pooler_output
      tag_embeddings.append(pooler_output)
    label_embeddings = torch.stack(tag_embeddings,dim=0)
    label_embeddings = label_embeddings.squeeze(1)
    return label_embeddings, full_label_name

    '''
    mutul(a,b) a和b维度是否一致的问题
    A.shape =（b,m,n)；B.shape = (b,n,k)
    torch.matmul(A,B) 结果shape为(b,m,k)
    '''

  def forward(self,input_ids,attention_mask,flag = True):
    if flag:
      label_representation = self.build_label_representation()[0].to(self.device)
      self.label_representation = label_representation.detach()
    else:
      label_representation = self.label_representation

    outputs = self.token_encoder(
        input_ids=input_ids,
        attention_mask = attention_mask
    )
                                  
    token_embeddings = outputs.last_hidden_state

    tag_lens,hidden_size = self.label_representation.shape

    current_batch_size  = token_embeddings.shape[0]

    label_embedding = self.label_representation.expand(current_batch_size,tag_lens,hidden_size)
    label_embeddings = label_embedding.transpose(2,1)
    matrix_embeddings = torch.matmul(token_embeddings,label_embeddings)
    softmax_embedding = nn.Softmax(dim=-1)(matrix_embeddings)
    label_indexs = torch.argmax(softmax_embedding,dim=-1)
    return matrix_embeddings,label_indexs



In [11]:
def produce_input_data(tokens,ner_tags,tokenizer,mask_number,probability):

  token_id= []
  ner_tags_id = []
  attention_mask = []
  # process data, do mask number, add [cls],[sep] tokens to the start and end of the tokens,
  #produce attention masks
  #produce targets matrix by padding -1
  for i in tokens:
    if(mask_number == True):
      i = random_mask_number(probability, i)
    text = ['[CLS]']+i+['[SEP]']
    token_id.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))
    
  all_pad_tokens = pad_sequences(token_id, maxlen=202, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=202, padding='post', truncating='post')

  for j in ner_tags:
    tags = [-1]+j+[-1]
    ner_tags_id.append(tags)
  all_pad_tags = pad_sequences(ner_tags_id, maxlen=202, padding='post', truncating='post',value=-1)

  glb_id = []
  for i in range(len(tokens)):
    glb_id.append(i)

  token_len = []
  for i in tokens:
    token_len.append(len(i))
 
  return{
      
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),
      'targets': torch.tensor(all_pad_tags, dtype=torch.long),
      'glb_id': torch.tensor(glb_id, dtype=torch.long),
      'token_len':torch.tensor(token_len, dtype=torch.long),
    }

In [12]:
class DataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      self.labels = data['targets']
      self.glb_id = data['glb_id']
      self.token_len = data['token_len']

    def __len__(self):

        return len(self.features)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.labels[index],self.glb_id[index],self.token_len[index]

In [13]:
def get_all_data(data,tokenizer,mask_number,max_len,probability,random_seed,shuffle:bool):
  #produce tokens , ner_tags
  tokens,ner_tags = data_preprocess(data,max_len)
  #using data_preprocess function to convert the data to a dict which include attention mask, inputs id, and targets
  data = produce_input_data(tokens,ner_tags,tokenizer,mask_number,probability)
  data = DataSequence(data)
  if shuffle == True:
    dataloader = DataLoader(data, num_workers=10, batch_size=8, shuffle=True, worker_init_fn=np.random.seed(random_seed))
  else:
    dataloader = DataLoader(data, num_workers=10, batch_size=8)

  return dataloader

In [14]:
sec_tokenizer = AutoTokenizer.from_pretrained("nlpaueb/sec-bert-base",do_lower_case=True)
finbert_tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone',do_lower_case=True)
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",do_lower_case=True)

Downloading:   0%|          | 0.00/263 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/568 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/221k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/533 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [15]:
class print_bad_case():
  def __init__(self,all_label_name):
    self.sentences = []
    self.true_label = []
    self.pred_label = []
    self.all_label_name = all_label_name

  def get_batch_badcase(self,glb_id,real_targets,pred_targets,token_lens,tokens):
    for id in glb_id:
      self.sentences.append([])
      for token in tokens[id]:
        self.sentences.append([token])

      
    for real_target,token_len in zip(real_targets,token_lens):
  
      self.true_label.append([])
      for token in real_target[1:token_len+1]:
        self.true_label.append(self.all_label_name[token.item()])

    for pred_target,token_len in zip(pred_targets,token_lens):

      self.pred_label.append([])
      for token in pred_target[1:token_len+1]:
        self.pred_label.append(self.all_label_name[token.item()])

  def generate_badcase_file(self):
    column_sentence = pd.DataFrame({'sentence':self.sentences})
    column_real_label = pd.DataFrame({'real_label':self.true_label})
    column_pred_label = pd.DataFrame({'pred_label':self.pred_label})
    
    df = pd.concat([column_sentence,column_real_label,column_pred_label],axis = 1)
    return df 
  def f1_calculation(self):

    true = [value for value in self.true_label if value != []]
    pred = [value for value in self.pred_label if value != []]

    f1 = f1_score([true], [pred],mode='strict',scheme=IOB2) 
    return f1
   

In [23]:
def train_and_eval(model, 
  train_loader, 
  validation_loader,
  train_tokens,
  validation_tokens,
  loss_fn, 
  optimizer, 
  all_label_name, 
  device,
  epoch,
  label_num,
  need_badcase_file:bool,
  model_name:str,
  database_name:str):
  model.train()
  f1_state = 0
  for epo in range(epoch):
    
    total_loss_train = 0
    total_loss_validation = 0


    # for print bad case using
    train_badcase = print_bad_case(all_label_name)
    validation_badcase = print_bad_case(all_label_name)
    
    
    for data in train_loader:
      
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      targets = data[2].to(device)
      glb_id = data[3].to(device)
      token_lens = data[4].to(device)


      matrix_embeddings,label_indexs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      # print(label_indexs.shape)
      # print(targets.shape)
    
      
      loss = loss_fn(matrix_embeddings.view(-1, label_num), targets.view(-1))
       
      #for printing bad case
      train_badcase.get_batch_badcase(glb_id,targets,label_indexs,token_lens,train_tokens)

      total_loss_train += loss.item()
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    model.eval()

    with torch.no_grad():
      for data in validation_loader:
    
        input_ids = data[0].to(device)
        attention_mask = data[1].to(device)
        targets = data[2].to(device)
        glb_id = data[3].to(device)
        token_lens = data[4].to(device)

        matrix_embeddings,label_indexs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        flag = False
        )
        
        loss = loss_fn(matrix_embeddings.view(-1, label_num), targets.view(-1))
        total_loss_validation += loss.item()


        # for print bad case
        validation_badcase.get_batch_badcase(glb_id,targets,label_indexs,token_lens,validation_tokens)
        
    
    #for print f1 using:
    train_f1 = train_badcase.f1_calculation()
    val_f1 = validation_badcase.f1_calculation()
    
    if(val_f1 > f1_state):
      f1_state = val_f1
      torch.save(model.state_dict(), model_name+'_'+database_name+'sou_state.bin')

        # for printing bad case using

      # ====================== start ===========================================
      if need_badcase_file == True:

        df_train = train_badcase.generate_badcase_file()
        df_validation = validation_badcase.generate_badcase_file()

      # ====================== end ===========================================

    val_loss = total_loss_validation / len(validation_loader)   
    train_loss = total_loss_train / len(train_loader)
    print('Epoch [{}] Train Loss: {:.4f}, Train F1_SCORE: {:.4f}, Valid Loss: {:.4f}, Valid F1_SCORE: {:.4f},'
                      .format(epo+1,train_loss, train_f1, val_loss, val_f1))
    
    
  if need_badcase_file == True:

    return df_train,df_validation


In [24]:
def test_prediction(model, 
  test_loader, 
  test_tokens,
  all_label_name,
  loss_fn, 
  optimizer,
  device,
  need_badcase_file:bool
  ):
  
  
  test_badcase = print_bad_case(all_label_name)


  model.eval()
  with torch.no_grad():
    for data in test_loader:
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      targets = data[2].to(device)
      glb_id = data[3].to(device)
      token_lens = data[4].to(device)
      matrix_embeddings,label_indexs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      
  
      #for printing bad case
      #====================== start =====================================
      test_badcase.get_batch_badcase(glb_id,targets,label_indexs,token_lens,test_tokens)

     
      # ==================================== end =============================
      
      
  test_f1 = test_badcase.f1_calculation()

  print('Test F1_SCORE: {:.4f}'.format(test_f1))
  # for bad case printing
  # ====================== start ===========================================
  
  if need_badcase_file == True:

    df_test = test_badcase.generate_badcase_file()
    return df_test
  
    

In [18]:

# ,'BusinessAcquisition','BusinessCombination','DebtInstrument','Equity','LineOfCredit','Loss','OperatingLease','Revenue','SharebasedCompensation','Stock','Tax','Transaction

In [30]:
souce_domain = ['Transaction']
multi_task_dataset = ['Stock','Revenue','Tax']
settings = [1,5,10,20]

In [31]:
#parameter settings 
for i in multi_task_dataset:
  for j in settings:
        
    Batch_size = 8
    dataset_name = i
    if j == 1:
      EPOCHS = 40
    else:
      EPOCHS = 20
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    loss_fn = nn.CrossEntropyLoss(ignore_index=-1).to(device)

    mask_number = False
    max_len = 200
    probability = 100
    random_seed = 42

    # model
    model_name = 'labert'
    few_shot = FewShot_NER(Batch_size,dataset_name).to(device)
    optimizer = torch.optim.Adam(few_shot.parameters(), lr=2e-5)
    need_badcase_file = True

    # load dataset
    os.chdir('/content/drive/MyDrive/latest_dataset/{}'.format(dataset_name))
    train = datasets.load_from_disk('{}_train_number'.format(dataset_name))
    test = datasets.load_from_disk('{}_test_number'.format(dataset_name))
    validation = datasets.load_from_disk('{}_validation_number'.format(dataset_name))

    train,_ = produce_database(train['tokens'],train['ner_tags'],j,few_shot.label_num)

    train_tokens,_ = data_preprocess(train,max_len)
    validation_tokens,_ = data_preprocess(validation,max_len)
    test_tokens,_ = data_preprocess(test,max_len)

    train_dataloader = get_all_data(train,finbert_tokenizer,mask_number,max_len,probability,random_seed,shuffle = False)
    validation_dataloader = get_all_data(validation,finbert_tokenizer,mask_number,max_len,probability,random_seed,shuffle = False)
    test_dataloader = get_all_data(test,finbert_tokenizer,mask_number,max_len,probability,random_seed,shuffle = False)

    #load source domain trained model
    os.chdir('/content/drive/MyDrive/latest_dataset/{}'.format(souce_domain[0]))
    model_path = '{}_{}_state.bin'.format(model_name,souce_domain[0]) 
    few_shot.load_state_dict(torch.load(model_path, map_location=torch.device("cuda:0")))
    os.chdir('/content/drive/MyDrive/latest_dataset/{}'.format(dataset_name))

    df_train,df_validation = train_and_eval(few_shot, 
        train_dataloader, 
        validation_dataloader,
        train_tokens,
        validation_tokens,
        loss_fn, 
        optimizer, 
        few_shot.idx2tags, 
        device,
        EPOCHS,
        few_shot.label_num,
        need_badcase_file,
        model_name,
        dataset_name)
    model_path = '{}_{}sou_state.bin'.format(model_name,dataset_name) 

    few_shot.load_state_dict(torch.load(model_path, map_location=torch.device("cuda:0")))

    df_test = test_prediction(few_shot, 
          test_dataloader, 
          test_tokens,
          few_shot.idx2tags,
          loss_fn, 
          optimizer,
          device,
          need_badcase_file)

    df_train = df_train.mask(df_train.applymap(str).eq('[]'))
    df_train =df_train.drop([0])


    df_validation = df_validation.mask(df_validation.applymap(str).eq('[]'))
    df_validation =df_validation.drop([0])

    df_test = df_test.mask(df_test.applymap(str).eq('[]'))
    df_test =df_test.drop([0])
    # label_semantic_result

    if not os.path.exists('/content/drive/MyDrive/source_domain/{}'.format(dataset_name)):
      os.mkdir('/content/drive/MyDrive/source_domain/{}'.format(dataset_name))
    os.chdir('/content/drive/MyDrive/source_domain/{}'.format(dataset_name))

    df_train.to_csv('{}_{}_train_bad_case.csv'.format(dataset_name,j),index=False)
    df_validation.to_csv('{}_{}_validation_bad_case.csv'.format(dataset_name,j),index=False)
    df_test.to_csv('{}_{}_test_bad_case.csv'.format(dataset_name,j),index=False)


Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

Epoch [1] Train Loss: 0.9924, Train F1_SCORE: 0.0286, Valid Loss: 0.4760, Valid F1_SCORE: 0.0000,
Epoch [2] Train Loss: 0.4320, Train F1_SCORE: 0.1622, Valid Loss: 0.2791, Valid F1_SCORE: 0.1138,
Epoch [3] Train Loss: 0.1977, Train F1_SCORE: 0.2828, Valid Loss: 0.2625, Valid F1_SCORE: 0.1530,
Epoch [4] Train Loss: 0.0967, Train F1_SCORE: 0.3604, Valid Loss: 0.2485, Valid F1_SCORE: 0.1885,
Epoch [5] Train Loss: 0.0555, Train F1_SCORE: 0.5741, Valid Loss: 0.2285, Valid F1_SCORE: 0.2110,
Epoch [6] Train Loss: 0.0343, Train F1_SCORE: 0.6981, Valid Loss: 0.2113, Valid F1_SCORE: 0.2257,
Epoch [7] Train Loss: 0.0236, Train F1_SCORE: 0.7619, Valid Loss: 0.1989, Valid F1_SCORE: 0.2344,
Epoch [8] Train Loss: 0.0175, Train F1_SCORE: 0.8713, Valid Loss: 0.1911, Valid F1_SCORE: 0.2411,
Epoch [9] Train Loss: 0.0135, Train F1_SCORE: 0.9307, Valid Loss: 0.1862, Valid F1_SCORE: 0.2463,
Epoch [10] Train Loss: 0.0105, Train F1_SCORE: 0.9600, Valid Loss: 0.1834, Valid F1_SCORE: 0.2503,
Epoch [11] Train Lo

Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

Epoch [1] Train Loss: 0.6479, Train F1_SCORE: 0.0317, Valid Loss: 0.3829, Valid F1_SCORE: 0.0431,
Epoch [2] Train Loss: 0.2277, Train F1_SCORE: 0.2196, Valid Loss: 0.1762, Valid F1_SCORE: 0.1858,
Epoch [3] Train Loss: 0.0903, Train F1_SCORE: 0.4327, Valid Loss: 0.1225, Valid F1_SCORE: 0.2636,
Epoch [4] Train Loss: 0.0455, Train F1_SCORE: 0.6780, Valid Loss: 0.1102, Valid F1_SCORE: 0.3681,
Epoch [5] Train Loss: 0.0265, Train F1_SCORE: 0.8070, Valid Loss: 0.1087, Valid F1_SCORE: 0.4044,
Epoch [6] Train Loss: 0.0166, Train F1_SCORE: 0.8949, Valid Loss: 0.1126, Valid F1_SCORE: 0.3824,
Epoch [7] Train Loss: 0.0118, Train F1_SCORE: 0.9521, Valid Loss: 0.1142, Valid F1_SCORE: 0.4236,
Epoch [8] Train Loss: 0.0077, Train F1_SCORE: 0.9704, Valid Loss: 0.1115, Valid F1_SCORE: 0.4239,
Epoch [9] Train Loss: 0.0054, Train F1_SCORE: 0.9777, Valid Loss: 0.1126, Valid F1_SCORE: 0.4371,
Epoch [10] Train Loss: 0.0044, Train F1_SCORE: 0.9926, Valid Loss: 0.1151, Valid F1_SCORE: 0.4371,
Epoch [11] Train Lo

Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

...

Epoch [1] Train Loss: 0.7198, Train F1_SCORE: 0.0100, Valid Loss: 0.3453, Valid F1_SCORE: 0.0615,
Epoch [2] Train Loss: 0.1727, Train F1_SCORE: 0.3077, Valid Loss: 0.1430, Valid F1_SCORE: 0.3309,
Epoch [3] Train Loss: 0.0695, Train F1_SCORE: 0.6415, Valid Loss: 0.1067, Valid F1_SCORE: 0.5045,
Epoch [4] Train Loss: 0.0279, Train F1_SCORE: 0.7925, Valid Loss: 0.0873, Valid F1_SCORE: 0.5899,
Epoch [5] Train Loss: 0.0148, Train F1_SCORE: 0.9091, Valid Loss: 0.0841, Valid F1_SCORE: 0.6276,
Epoch [6] Train Loss: 0.0122, Train F1_SCORE: 0.9320, Valid Loss: 0.0836, Valid F1_SCORE: 0.6357,
Epoch [7] Train Loss: 0.0082, Train F1_SCORE: 0.9712, Valid Loss: 0.0807, Valid F1_SCORE: 0.6451,
Epoch [8] Train Loss: 0.0065, Train F1_SCORE: 0.9808, Valid Loss: 0.0805, Valid F1_SCORE: 0.6465,
Epoch [9] Train Loss: 0.0055, Train F1_SCORE: 0.9904, Valid Loss: 0.0813, Valid F1_SCORE: 0.6466,
Epoch [10] Train Loss: 0.0048, Train F1_SCORE: 0.9904, Valid Loss: 0.0821, Valid F1_SCORE: 0.6408,
Epoch [11] Train Lo

Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

Epoch [1] Train Loss: 0.5259, Train F1_SCORE: 0.0136, Valid Loss: 0.3262, Valid F1_SCORE: 0.2228,
Epoch [2] Train Loss: 0.1177, Train F1_SCORE: 0.3922, Valid Loss: 0.1097, Valid F1_SCORE: 0.4864,
Epoch [3] Train Loss: 0.0499, Train F1_SCORE: 0.5475, Valid Loss: 0.0805, Valid F1_SCORE: 0.6401,
Epoch [4] Train Loss: 0.0252, Train F1_SCORE: 0.8103, Valid Loss: 0.0733, Valid F1_SCORE: 0.6745,
Epoch [5] Train Loss: 0.0128, Train F1_SCORE: 0.9101, Valid Loss: 0.0707, Valid F1_SCORE: 0.6640,
Epoch [6] Train Loss: 0.0092, Train F1_SCORE: 0.9408, Valid Loss: 0.0704, Valid F1_SCORE: 0.6813,
Epoch [7] Train Loss: 0.0074, Train F1_SCORE: 0.9494, Valid Loss: 0.0705, Valid F1_SCORE: 0.6923,
Epoch [8] Train Loss: 0.0062, Train F1_SCORE: 0.9607, Valid Loss: 0.0704, Valid F1_SCORE: 0.6962,
Epoch [9] Train Loss: 0.0054, Train F1_SCORE: 0.9719, Valid Loss: 0.0706, Valid F1_SCORE: 0.6969,
Epoch [10] Train Loss: 0.0047, Train F1_SCORE: 0.9775, Valid Loss: 0.0710, Valid F1_SCORE: 0.6993,
Epoch [11] Train Lo

Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

Epoch [1] Train Loss: 0.4679, Train F1_SCORE: 0.0967, Valid Loss: 0.1870, Valid F1_SCORE: 0.3108,
Epoch [2] Train Loss: 0.0830, Train F1_SCORE: 0.5055, Valid Loss: 0.0760, Valid F1_SCORE: 0.6348,
Epoch [3] Train Loss: 0.0385, Train F1_SCORE: 0.6703, Valid Loss: 0.0696, Valid F1_SCORE: 0.6409,
Epoch [4] Train Loss: 0.0244, Train F1_SCORE: 0.7840, Valid Loss: 0.0589, Valid F1_SCORE: 0.7390,
Epoch [5] Train Loss: 0.0326, Train F1_SCORE: 0.8237, Valid Loss: 0.0628, Valid F1_SCORE: 0.7037,
Epoch [6] Train Loss: 0.0116, Train F1_SCORE: 0.9187, Valid Loss: 0.0663, Valid F1_SCORE: 0.7103,
Epoch [7] Train Loss: 0.0090, Train F1_SCORE: 0.9295, Valid Loss: 0.0585, Valid F1_SCORE: 0.7552,
Epoch [8] Train Loss: 0.0062, Train F1_SCORE: 0.9644, Valid Loss: 0.0604, Valid F1_SCORE: 0.7534,
Epoch [9] Train Loss: 0.0047, Train F1_SCORE: 0.9725, Valid Loss: 0.0619, Valid F1_SCORE: 0.7524,
Epoch [10] Train Loss: 0.0039, Train F1_SCORE: 0.9835, Valid Loss: 0.0612, Valid F1_SCORE: 0.7599,
Epoch [11] Train Lo

In [ ]:
93 84 81

In [ ]:
# ,'BusinessAcquisition','BusinessCombination','DebtInstrument','Equity','LineOfCredit','Loss','OperatingLease','Revenue','SharebasedCompensation','Stock','Tax','Transaction


In [34]:
souce_domain = ['Transaction']
multi_task_dataset = ['DebtInstrument','OperatingLease','LineOfCredit','Equity']
settings = [1,5,10,20]

In [ ]:
#parameter settings 
for i in multi_task_dataset:
  for j in settings:
        
    Batch_size = 8
    dataset_name = i
    if j == 1:
      EPOCHS = 40
    else:
      EPOCHS = 20
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    loss_fn = nn.CrossEntropyLoss(ignore_index=-1).to(device)

    mask_number = False
    max_len = 200
    probability = 100
    random_seed = 42

    # model
    model_name = 'labert'
    few_shot = FewShot_NER(Batch_size,dataset_name).to(device)
    optimizer = torch.optim.Adam(few_shot.parameters(), lr=2e-5)
    need_badcase_file = True

    # load dataset
    os.chdir('/content/drive/MyDrive/latest_dataset/{}'.format(dataset_name))
    train = datasets.load_from_disk('{}_train_number'.format(dataset_name))
    test = datasets.load_from_disk('{}_test_number'.format(dataset_name))
    validation = datasets.load_from_disk('{}_validation_number'.format(dataset_name))

    train,_ = produce_database(train['tokens'],train['ner_tags'],j,few_shot.label_num)

    train_tokens,_ = data_preprocess(train,max_len)
    validation_tokens,_ = data_preprocess(validation,max_len)
    test_tokens,_ = data_preprocess(test,max_len)

    train_dataloader = get_all_data(train,finbert_tokenizer,mask_number,max_len,probability,random_seed,shuffle = False)
    validation_dataloader = get_all_data(validation,finbert_tokenizer,mask_number,max_len,probability,random_seed,shuffle = False)
    test_dataloader = get_all_data(test,finbert_tokenizer,mask_number,max_len,probability,random_seed,shuffle = False)

    #load source domain trained model
    os.chdir('/content/drive/MyDrive/latest_dataset/{}'.format(souce_domain[0]))
    model_path = '{}_{}_state.bin'.format(model_name,souce_domain[0]) 
    few_shot.load_state_dict(torch.load(model_path, map_location=torch.device("cuda:0")))
    os.chdir('/content/drive/MyDrive/latest_dataset/{}'.format(dataset_name))

    df_train,df_validation = train_and_eval(few_shot, 
        train_dataloader, 
        validation_dataloader,
        train_tokens,
        validation_tokens,
        loss_fn, 
        optimizer, 
        few_shot.idx2tags, 
        device,
        EPOCHS,
        few_shot.label_num,
        need_badcase_file,
        model_name,
        dataset_name)
    model_path = '{}_{}sou_state.bin'.format(model_name,dataset_name) 

    few_shot.load_state_dict(torch.load(model_path, map_location=torch.device("cuda:0")))

    df_test = test_prediction(few_shot, 
          test_dataloader, 
          test_tokens,
          few_shot.idx2tags,
          loss_fn, 
          optimizer,
          device,
          need_badcase_file)

    df_train = df_train.mask(df_train.applymap(str).eq('[]'))
    df_train =df_train.drop([0])


    df_validation = df_validation.mask(df_validation.applymap(str).eq('[]'))
    df_validation =df_validation.drop([0])

    df_test = df_test.mask(df_test.applymap(str).eq('[]'))
    df_test =df_test.drop([0])
    # label_semantic_result

    if not os.path.exists('/content/drive/MyDrive/source_domain/{}'.format(dataset_name)):
      os.mkdir('/content/drive/MyDrive/source_domain/{}'.format(dataset_name))
    os.chdir('/content/drive/MyDrive/source_domain/{}'.format(dataset_name))

    df_train.to_csv('{}_{}_train_bad_case.csv'.format(dataset_name,j),index=False)
    df_validation.to_csv('{}_{}_validation_bad_case.csv'.format(dataset_name,j),index=False)
    df_test.to_csv('{}_{}_test_bad_case.csv'.format(dataset_name,j),index=False)


Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

Epoch [1] Train Loss: 0.9092, Train F1_SCORE: 0.0000, Valid Loss: 0.5405, Valid F1_SCORE: 0.0379,
Epoch [2] Train Loss: 0.3875, Train F1_SCORE: 0.0702, Valid Loss: 0.3337, Valid F1_SCORE: 0.0924,
Epoch [3] Train Loss: 0.2061, Train F1_SCORE: 0.1389, Valid Loss: 0.3300, Valid F1_SCORE: 0.1159,
Epoch [4] Train Loss: 0.0967, Train F1_SCORE: 0.4308, Valid Loss: 0.2790, Valid F1_SCORE: 0.1615,
Epoch [5] Train Loss: 0.0458, Train F1_SCORE: 0.6250, Valid Loss: 0.2388, Valid F1_SCORE: 0.1991,
Epoch [6] Train Loss: 0.0263, Train F1_SCORE: 0.6875, Valid Loss: 0.2152, Valid F1_SCORE: 0.2364,
Epoch [7] Train Loss: 0.0183, Train F1_SCORE: 0.7302, Valid Loss: 0.2023, Valid F1_SCORE: 0.2636,
Epoch [8] Train Loss: 0.0128, Train F1_SCORE: 0.8438, Valid Loss: 0.1952, Valid F1_SCORE: 0.2882,
Epoch [9] Train Loss: 0.0089, Train F1_SCORE: 0.9062, Valid Loss: 0.1913, Valid F1_SCORE: 0.3082,
Epoch [10] Train Loss: 0.0064, Train F1_SCORE: 0.9538, Valid Loss: 0.1892, Valid F1_SCORE: 0.3231,
Epoch [11] Train Lo

Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

Epoch [1] Train Loss: 0.7293, Train F1_SCORE: 0.0153, Valid Loss: 0.4957, Valid F1_SCORE: 0.0832,
Epoch [2] Train Loss: 0.2260, Train F1_SCORE: 0.1710, Valid Loss: 0.1866, Valid F1_SCORE: 0.2864,
Epoch [3] Train Loss: 0.0639, Train F1_SCORE: 0.4962, Valid Loss: 0.1499, Valid F1_SCORE: 0.3311,
Epoch [4] Train Loss: 0.0524, Train F1_SCORE: 0.6693, Valid Loss: 0.1724, Valid F1_SCORE: 0.4368,
Epoch [5] Train Loss: 0.0283, Train F1_SCORE: 0.7958, Valid Loss: 0.1403, Valid F1_SCORE: 0.4296,
Epoch [6] Train Loss: 0.0136, Train F1_SCORE: 0.8506, Valid Loss: 0.1375, Valid F1_SCORE: 0.4285,
Epoch [7] Train Loss: 0.0081, Train F1_SCORE: 0.9179, Valid Loss: 0.1252, Valid F1_SCORE: 0.4852,
Epoch [8] Train Loss: 0.0059, Train F1_SCORE: 0.9259, Valid Loss: 0.1239, Valid F1_SCORE: 0.5003,
Epoch [9] Train Loss: 0.0044, Train F1_SCORE: 0.9520, Valid Loss: 0.1240, Valid F1_SCORE: 0.5045,
Epoch [10] Train Loss: 0.0035, Train F1_SCORE: 0.9742, Valid Loss: 0.1248, Valid F1_SCORE: 0.5060,
Epoch [11] Train Lo

Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

Epoch [1] Train Loss: 0.9538, Train F1_SCORE: 0.0299, Valid Loss: 0.3945, Valid F1_SCORE: 0.2292,
Epoch [2] Train Loss: 0.3503, Train F1_SCORE: 0.2463, Valid Loss: 0.4592, Valid F1_SCORE: 0.2294,
Epoch [3] Train Loss: 0.2152, Train F1_SCORE: 0.3789, Valid Loss: 0.1679, Valid F1_SCORE: 0.2974,
Epoch [4] Train Loss: 0.1289, Train F1_SCORE: 0.5161, Valid Loss: 0.1733, Valid F1_SCORE: 0.4539,
Epoch [5] Train Loss: 0.0511, Train F1_SCORE: 0.7111, Valid Loss: 0.1322, Valid F1_SCORE: 0.4497,
Epoch [6] Train Loss: 0.0258, Train F1_SCORE: 0.8194, Valid Loss: 0.1181, Valid F1_SCORE: 0.5201,
Epoch [7] Train Loss: 0.0151, Train F1_SCORE: 0.8757, Valid Loss: 0.1140, Valid F1_SCORE: 0.5211,
Epoch [8] Train Loss: 0.0123, Train F1_SCORE: 0.9161, Valid Loss: 0.1145, Valid F1_SCORE: 0.5379,
Epoch [9] Train Loss: 0.0090, Train F1_SCORE: 0.9458, Valid Loss: 0.1145, Valid F1_SCORE: 0.5421,
Epoch [10] Train Loss: 0.0072, Train F1_SCORE: 0.9655, Valid Loss: 0.1158, Valid F1_SCORE: 0.5468,
Epoch [11] Train Lo

Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

Epoch [1] Train Loss: 0.7789, Train F1_SCORE: 0.0602, Valid Loss: 0.2752, Valid F1_SCORE: 0.0805,
Epoch [2] Train Loss: 0.2385, Train F1_SCORE: 0.3092, Valid Loss: 0.1491, Valid F1_SCORE: 0.2561,
Epoch [3] Train Loss: 0.1104, Train F1_SCORE: 0.5226, Valid Loss: 0.1056, Valid F1_SCORE: 0.5387,
Epoch [4] Train Loss: 0.0663, Train F1_SCORE: 0.6789, Valid Loss: 0.1531, Valid F1_SCORE: 0.5666,
Epoch [5] Train Loss: 0.0478, Train F1_SCORE: 0.7780, Valid Loss: 0.2436, Valid F1_SCORE: 0.5530,
Epoch [6] Train Loss: 0.0433, Train F1_SCORE: 0.8217, Valid Loss: 0.4226, Valid F1_SCORE: 0.5216,
Epoch [7] Train Loss: 0.0585, Train F1_SCORE: 0.8548, Valid Loss: 0.3250, Valid F1_SCORE: 0.5840,
Epoch [8] Train Loss: 0.0450, Train F1_SCORE: 0.8717, Valid Loss: 0.3621, Valid F1_SCORE: 0.5849,
Epoch [9] Train Loss: 0.0428, Train F1_SCORE: 0.8654, Valid Loss: 0.1471, Valid F1_SCORE: 0.5902,
Epoch [10] Train Loss: 0.0236, Train F1_SCORE: 0.8825, Valid Loss: 0.1482, Valid F1_SCORE: 0.5854,
Epoch [11] Train Lo